# Diferenciação automática

Derivar a função $y = 2\mathbf{x}^{\top}\mathbf{x}$ com respeito ao vetor coluna $\mathbf{x}$

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

Antes de calcularmos o gradiente de$y$ em relação a $\mathbf{x}$,
precisamos armazena-lo

In [2]:
x.requires_grad_(True)
x.grad

Então calcularemos $y$

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

Podemos calcular automaticamente o gradiente de `y`
com relação a cada componente de `x`

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

O gradiente da função $y = 2\mathbf{x}^{\top}\mathbf{x}$
em relação a $\mathbf{x}$ should be $4\mathbf{x}$

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

Agora calculamos outra função de `x`

In [6]:
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

Em *Deep Learning*
quando estamos retrocedendo um vetor,
nossa intenção é
a soma das derivadas parciais
calculado individualmente para cada exemplo

In [7]:
x.grad.zero_()
y = x * x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

Mover alguns cálculos
fora do gráfico computacional registrado

In [8]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

Mesmo se
uma função
requer muito trabalho com o uso do  *Python Control Flow*
ainda podemos calcular o gradiente da variável resultante

In [12]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(False)